In [200]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import zscore
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,MinMaxScaler
from sklearn.decomposition import PCA

In [182]:
df=pd.read_csv("heart.csv")
df.sample(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
868,51,M,NAP,110,175,0,Normal,123,N,0.6,Up,0
67,32,M,ATA,110,225,0,Normal,184,N,0.0,Up,0
720,48,M,ASY,124,274,0,LVH,166,N,0.5,Flat,1
58,54,M,ASY,150,365,0,ST,134,N,1.0,Up,0
436,58,M,ASY,116,0,0,Normal,124,N,1.0,Up,1
908,63,M,ASY,140,187,0,LVH,144,Y,4.0,Up,1
344,51,M,ASY,120,0,1,Normal,104,N,0.0,Flat,1
822,58,M,NAP,105,240,0,LVH,154,Y,0.6,Flat,0
700,42,M,TA,148,244,0,LVH,178,N,0.8,Up,0
717,49,F,ASY,130,269,0,Normal,163,N,0.0,Up,0


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [184]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [185]:
cols=['RestingBP','Cholesterol','MaxHR','Oldpeak']
for i in cols:
    df[f'Z-score_{i}'] = zscore(df[i])


In [186]:
df.head(10)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Z-score_RestingBP,Z-score_Cholesterol,Z-score_MaxHR,Z-score_Oldpeak
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,0.410909,0.825070,1.382928,-0.832432
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,1.491752,-0.171961,0.754157,0.105664
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-0.129513,0.770188,-1.525138,-0.832432
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0.302825,0.139040,-1.132156,0.574711
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.951331,-0.034755,-0.581981,-0.832432
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0,-0.669935,1.282424,1.304332,-0.832432
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0,-0.129513,0.349422,1.304332,-0.832432
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0,-1.210356,0.084157,0.203982,-0.832432
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1,0.410909,0.075010,-0.267596,0.574711
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0,-0.669935,0.779335,-0.660578,-0.832432


In [187]:
df.shape

(918, 16)

In [188]:
indices=df[(df['Z-score_RestingBP'].abs() > 3) | (df['Z-score_Cholesterol'].abs() > 3) | (df['Z-score_Oldpeak'].abs() > 3)].index

In [189]:
df.drop(indices,axis='index',inplace=True)
df.shape

(900, 16)

In [190]:
df.drop(['Z-score_RestingBP','Z-score_Cholesterol','Z-score_MaxHR','Z-score_Oldpeak'],axis='columns',inplace =True)
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [191]:
df['ST_Slope'].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [192]:
x=df.drop(["HeartDisease"],axis='columns')
y=df['HeartDisease']

In [193]:
ct=ColumnTransformer(transformers=[
    ('tnf1',OrdinalEncoder(categories=[['Up','Flat','Down']]),['ST_Slope']),
    ('tnf2',OneHotEncoder(sparse_output=False,drop='first'),['Sex','ChestPainType','ExerciseAngina','RestingECG'])
],remainder='passthrough'
)
x=ct.fit_transform(x)
x[:10,:]

array([[  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  40. ,
        140. , 289. ,   0. , 172. ,   0. ],
       [  1. ,   0. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  49. ,
        160. , 180. ,   0. , 156. ,   1. ],
       [  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,  37. ,
        130. , 283. ,   0. ,  98. ,   0. ],
       [  1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,  48. ,
        138. , 214. ,   0. , 108. ,   1.5],
       [  0. ,   1. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  54. ,
        150. , 195. ,   0. , 122. ,   0. ],
       [  0. ,   1. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,  39. ,
        120. , 339. ,   0. , 170. ,   0. ],
       [  0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  45. ,
        130. , 237. ,   0. , 170. ,   0. ],
       [  0. ,   1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  54. ,
        110. , 208. ,   0. , 142. ,   0. ],
       [  1. ,   1. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,  37. ,
        1

In [194]:
x.shape

(900, 14)

In [197]:
scaler=MinMaxScaler()
x_scale=scaler.fit_transform(x)
x_scale[:10,:]

array([[0.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.24489796, 0.57142857,
        0.55791506, 0.        , 0.78873239, 0.33333333],
       [0.5       , 0.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.42857143, 0.76190476,
        0.34749035, 0.        , 0.67605634, 0.5       ],
       [0.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.18367347, 0.47619048,
        0.54633205, 0.        , 0.26760563, 0.33333333],
       [0.5       , 0.        , 0.        , 0.        , 0.        ,
        1.        , 1.        , 0.        , 0.40816327, 0.55238095,
        0.41312741, 0.        , 0.33802817, 0.58333333],
       [0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.53061224, 0.66666667,
        0.37644788, 0.        , 0.43661972, 0.33333333],
       [0.        , 1.        , 0. 

In [198]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'Multinomial_NB' : {
        'model': MultinomialNB(),
        'params': {
        
        }
    },

}

In [199]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_scale,y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
final_df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
final_df

c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SA

,model,best_score,best_params
0,svm,0.820000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.796667,{'n_estimators': 10}
2,logistic_regression,0.812222,{'C': 1}
3,Multinomial_NB,0.815556,{}


In [ ]:
pca=PCA(0.95)
pca.fit_transform(x_scale)


In [201]:
scores2 = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_scale,y)
    scores2.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
final_df2 = pd.DataFrame(scores2,columns=['model','best_score','best_params'])
final_df2

c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SARKAR\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\SHUBHAM SA

,model,best_score,best_params
0,svm,0.820000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.820000,{'n_estimators': 10}
2,logistic_regression,0.812222,{'C': 1}
3,Multinomial_NB,0.815556,{}
